##### Copyright 2020 Die TensorFlow-Autoren.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Einführung in Farbverläufe und automatische Differenzierung

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Ansicht auf TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> In Google Colab ausführen</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Notizbuch herunterladen</a></td>
</table>

## Automatische Differenzierung und Farbverläufe

[Die automatische Differenzierung](https://en.wikipedia.org/wiki/Automatic_differentiation) ist nützlich für die Implementierung von Algorithmen für maschinelles Lernen, z. B. [Backpropagation](https://en.wikipedia.org/wiki/Backpropagation) zum Trainieren neuronaler Netze.

In diesem Handbuch werden Möglichkeiten erläutert, wie Sie mit TensorFlow Farbverläufe berechnen können, insbesondere bei eifriger Ausführung.

## Konfiguration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

## Gradienten berechnen

Um automatisch zu unterscheiden, muss sich TensorFlow merken, welche Vorgänge während des *Vorwärtsdurchlaufs* in welcher Reihenfolge ablaufen. Während des *Rückwärtsdurchlaufs* durchläuft TensorFlow diese Liste von Operationen in umgekehrter Reihenfolge, um Gradienten zu berechnen.

## Verlaufsbänder

TensorFlow bietet die [tf.GradientTape-](https://www.tensorflow.org/api_docs/python/tf/GradientTape) API zur automatischen Differenzierung. das heißt, Berechnen des Gradienten einer Berechnung in Bezug auf einige Eingaben, normalerweise `tf.Variable` s. TensorFlow "zeichnet" relevante Operationen, die im Kontext eines `tf.GradientTape` auf ein "Band" auf. TensorFlow verwendet dann das Band die Gradienten einer „aufgezeichnet“ Berechnung verwendet berechnen [Rückwärtsmodus Differenzierung](https://en.wikipedia.org/wiki/Automatic_differentiation) .

Hier ist ein einfaches Beispiel:

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2

Wenn Sie einige Vorgänge aufgezeichnet haben, verwenden Sie `GradientTape.gradient(target, sources)` , um den Gradienten eines Ziels (häufig ein Verlust) relativ zu einer Quelle (häufig die Variablen des Modells) zu berechnen.

In [ ]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

Im obigen Beispiel werden Skalare verwendet, aber `tf.GradientTape` funktioniert auf jedem Tensor genauso einfach:

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

To get the gradient of `y` with respect to both variables, you can pass both as sources to the `gradient` method. The tape is flexible about how sources are passed and will accept any nested combination of lists or dictionaries and return the gradient structured the same way (see `tf.nest`).

In [ ]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

Der Gradient in Bezug auf jede Quelle hat die Form der Quelle:

In [ ]:
print(w.shape)
print(dl_dw.shape)

Hier ist noch einmal die Gradientenberechnung, diesmal mit einem Variablenwörterbuch:

In [ ]:
my_vars = {
    'w': tf.Variable(tf.random.normal((3, 2)), name='w'),
    'b': tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
}

grad = tape.gradient(loss, my_vars)
grad['b']

## Farbverläufe in Bezug auf ein Modell

Es ist üblich, `tf.Variables` in einem `tf.Module` oder einer seiner Unterklassen ( `layers.Layer` , `keras.Model` ) zum [Checkpointing](checkpoint.ipynb) und [Exportieren](saved_model.ipynb) zu sammeln.

In den meisten Fällen möchten Sie Gradienten in Bezug auf die trainierbaren Variablen eines Modells berechnen. Da alle Unterklassen von `tf.Module` ihre Variablen in der `Module.trainable_variables` Eigenschaft aggregieren, können Sie diese Verläufe in wenigen Codezeilen berechnen: 

In [ ]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)

In [ ]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape: {g.shape}')

<a id="watches"></a>

## Steuern, was das Band sieht

Das Standardverhalten besteht darin, alle Vorgänge nach dem Zugriff auf eine trainierbare `tf.Variable` . Die Gründe dafür sind:

- Das Band muss wissen, welche Operationen im Vorwärtsdurchlauf aufgezeichnet werden müssen, um die Gradienten im Rückwärtsdurchlauf zu berechnen.
- Das Band enthält Verweise auf Zwischenausgaben, sodass Sie keine unnötigen Vorgänge aufzeichnen möchten.
- Der häufigste Anwendungsfall besteht darin, den Gradienten eines Verlusts in Bezug auf alle trainierbaren Variablen eines Modells zu berechnen.

Im Folgenden wird beispielsweise kein Gradient berechnet, da der `tf.Tensor` standardmäßig nicht "überwacht" wird und die `tf.Variable` nicht trainierbar ist:

In [ ]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

Sie können die vom Band überwachten Variablen mit der Methode `GradientTape.watched_variables` auflisten:

In [ ]:
[var.name for var in tape.watched_variables()]

`tf.GradientTape` bietet Hooks, mit denen der Benutzer steuern kann, was überwacht wird oder nicht.

Um Farbverläufe in Bezug auf einen `tf.Tensor` , müssen Sie `GradientTape.watch(x)` aufrufen:

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x**2

# dy = 2x * dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

Conversely, to disable the default behavior of watching all `tf.Variables`, set `watch_accessed_variables=False` when creating the gradient tape. This calculation uses two variables, but only connects the gradient for one of the variables:

In [ ]:
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

Da `GradientTape.watch` auf `x0` nicht aufgerufen wurde, wird kein Gradient in Bezug darauf berechnet:

In [ ]:
# dy = 2x * dx
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print('dy/dx0:', grad['x0'])
print('dy/dx1:', grad['x1'].numpy())

## Zwischenergebnisse

You can also request gradients of the output with respect to intermediate values computed inside the `tf.GradientTape` context.

In [ ]:
x = tf.constant(3.0)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = x * x
  z = y * y

# Use the tape to compute the gradient of z with respect to the
# intermediate value y.
# dz_dx = 2 * y, where y = x ** 2
print(tape.gradient(z, y).numpy())

Standardmäßig werden die von einem `GradientTape` gehaltenen Ressourcen freigegeben, sobald die `GradientTape.gradient()` -Methode aufgerufen wird. Erstellen Sie ein `persistent` Verlaufsband, um mehrere Verläufe über dieselbe Berechnung zu berechnen. Dies ermöglicht mehrere Aufrufe der `gradient()` -Methode, wenn Ressourcen freigegeben werden, wenn das Bandobjekt durch Müll gesammelt wird. Beispielsweise:

In [ ]:
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy())  # 108.0 (4 * x**3 at x = 3)
print(tape.gradient(y, x).numpy())  # 6.0 (2 * x)

In [ ]:
del tape   # Drop the reference to the tape

## Hinweise zur Leistung

- Das Ausführen von Operationen in einem Gradientenbandkontext ist mit einem winzigen Aufwand verbunden. Für die eifrigste Ausführung sind die Kosten nicht spürbar. Sie sollten jedoch den Bandkontext nur in den Bereichen verwenden, in denen dies erforderlich ist.

- Verlaufsbänder verwenden Speicher, um Zwischenergebnisse, einschließlich Ein- und Ausgänge, zur Verwendung während des Rückwärtsdurchlaufs zu speichern.

    For efficiency, some ops (like `ReLU`) don't need to keep their intermediate results and they are pruned during the forward pass. However, if you use `persistent=True` on your tape, *nothing is discarded* and your peak memory usage will be higher.

## Gradienten nicht skalarer Ziele

Ein Gradient ist im Grunde eine Operation auf einem Skalar.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient(y0, x).numpy())
print(tape.gradient(y1, x).numpy())

Wenn Sie also nach dem Gradienten mehrerer Ziele fragen, lautet das Ergebnis für jede Quelle:

- Der Gradient der Summe der Ziele oder äquivalent
- Die Summe der Gradienten jedes Ziels.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient({'y0': y0, 'y1': y1}, x).numpy())

In ähnlicher Weise wird der Gradient der Summe berechnet, wenn die Ziele nicht skalar sind.

In [ ]:
x = tf.Variable(2.)

with tf.GradientTape() as tape:
  y = x * [3., 4.]

print(tape.gradient(y, x).numpy())

Dies macht es einfach, den Gradienten der Summe einer Sammlung von Verlusten oder den Gradienten der Summe einer elementweisen Verlustberechnung zu nehmen.

Wenn Sie für jeden Gegenstand einen eigenen Farbverlauf benötigen, lesen Sie [Jacobians](advanced_autodiff.ipynb#jacobians) .

In einigen Fällen können Sie den Jacobian überspringen. Für eine elementweise Berechnung gibt der Gradient der Summe die Ableitung jedes Elements in Bezug auf sein Eingabeelement an, da jedes Element unabhängig ist:

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.nn.sigmoid(x)

dy_dx = tape.gradient(y, x)

In [ ]:
plt.plot(x, y, label='y')
plt.plot(x, dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

## Kontrollfluss

Da Bänder Operationen aufzeichnen, während sie ausgeführt werden, wird der Python-Steuerungsfluss (z. B. `if` s und `while` s) natürlich behandelt.

Hier wird für jeden Zweig eines `if` eine andere Variable verwendet. Der Verlauf stellt nur eine Verbindung zu der verwendeten Variablen her:

In [ ]:
x = tf.constant(1.0)

v0 = tf.Variable(2.0)
v1 = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  if x > 0.0:
    result = v0
  else:
    result = v1**2 

dv0, dv1 = tape.gradient(result, [v0, v1])

print(dv0)
print(dv1)

Denken Sie daran, dass die Steueranweisungen selbst nicht differenzierbar sind und daher für gradientenbasierte Optimierer unsichtbar sind.

Abhängig vom Wert von `x` im obigen Beispiel zeichnet das Band entweder `result = v0` oder `result = v1**2` . Der Gradient in Bezug auf `x` ist immer `None` .

In [ ]:
dx = tape.gradient(result, x)

print(dx)

## Einen Gradienten von `None`

Wenn ein Ziel nicht mit einer Quelle verbunden ist, erhalten Sie einen Gradienten von `None` .


In [ ]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y * y
print(tape.gradient(z, x))

Hier ist `z` offensichtlich nicht mit `x` , aber es gibt mehrere weniger offensichtliche Möglichkeiten, wie ein Gradient getrennt werden kann.

### 1. Ersetzen Sie eine Variable durch einen Tensor.

Im Abschnitt ["Steuern, was das Band sieht" haben](#watches) Sie gesehen, dass das Band automatisch einen `tf.Variable` aber keinen `tf.Tensor` .

Ein häufiger Fehler besteht darin, versehentlich eine `tf.Variable` durch eine `tf.Tensor` zu ersetzen, anstatt `Variable.assign` zum Aktualisieren der `tf.Variable` . Hier ist ein Beispiel:

In [ ]:
x = tf.Variable(2.0)

for epoch in range(2):
  with tf.GradientTape() as tape:
    y = x+1

  print(type(x).__name__, ":", tape.gradient(y, x))
  x = x + 1   # This should be `x.assign_add(1)`

### 2. Berechnete außerhalb von TensorFlow

Das Band kann den Verlaufspfad nicht aufzeichnen, wenn die Berechnung TensorFlow beendet. Beispielsweise:

In [ ]:
x = tf.Variable([[1.0, 2.0],
                 [3.0, 4.0]], dtype=tf.float32)

with tf.GradientTape() as tape:
  x2 = x**2

  # This step is calculated with NumPy
  y = np.mean(x2, axis=0)

  # Like most ops, reduce_mean will cast the NumPy array to a constant tensor
  # using `tf.convert_to_tensor`.
  y = tf.reduce_mean(y, axis=0)

print(tape.gradient(y, x))

### 3. Gradienten durch eine Ganzzahl oder Zeichenfolge geführt

Ganzzahlen und Zeichenfolgen sind nicht differenzierbar. Wenn ein Berechnungspfad diese Datentypen verwendet, gibt es keinen Farbverlauf.

Nobody expects strings to be differentiable, but it's easy to accidentally create an `int` constant or variable if you don't specify the `dtype`.

In [ ]:
# The x0 variable has an `int` dtype.
x = tf.Variable([[2, 2],
                 [2, 2]])

with tf.GradientTape() as tape:
  # The path to x1 is blocked by the `int` dtype here.
  y = tf.cast(x, tf.float32)
  y = tf.reduce_sum(x)

print(tape.gradient(y, x))

TensorFlow wechselt nicht automatisch zwischen Typen. In der Praxis wird daher häufig ein Typfehler anstelle eines fehlenden Verlaufs angezeigt.

### 5. Gradienten durch ein zustandsbehaftetes Objekt geführt

Zustand stoppt Farbverläufe. Wenn Sie von einem statusbehafteten Objekt lesen, kann das Band nur den aktuellen Status sehen, nicht den Verlauf, der dazu geführt hat.

Ein `tf.Tensor` ist unveränderlich. Sie können einen Tensor nach seiner Erstellung nicht mehr ändern. Es hat einen *Wert* , aber keinen *Zustand* . Alle bisher diskutierten Operationen sind ebenfalls zustandslos: Die Ausgabe eines `tf.matmul` hängt nur von seinen Eingaben ab.

Eine `tf.Variable` hat einen internen Zustand, ihren Wert. Wenn Sie die Variable verwenden, wird der Status gelesen. Es ist normal, einen Gradienten in Bezug auf eine Variable zu berechnen, aber der Zustand der Variablen verhindert, dass Gradientenberechnungen weiter zurückgehen. Beispielsweise:


In [ ]:
x0 = tf.Variable(3.0)
x1 = tf.Variable(0.0)

with tf.GradientTape() as tape:
  # Update x1 = x1 + x0.
  x1.assign_add(x0)
  # The tape starts recording from x1.
  y = x1**2   # y = (x1 + x0)**2

# This doesn't work.
print(tape.gradient(y, x0))   #dy/dx0 = 2*(x1 + x2)

Similarly `tf.data.Dataset` iterators and `tf.queue`s are stateful, and will stop all gradients on tensors that pass through them.

## Kein Farbverlauf registriert

Einige `tf.Operation` werden **als nicht differenzierbar registriert** und geben `None` . Andere haben **keinen Gradienten registriert** .

The [tf.raw_ops](https://www.tensorflow.org/api_docs/python/tf/raw_ops) page shows which low-level ops have gradients registered.

Wenn Sie versuchen, einen Farbverlauf durch eine Float-Operation zu ziehen, bei der kein Farbverlauf registriert ist, gibt das Band einen Fehler aus, anstatt stillschweigend `None` . Auf diese Weise wissen Sie, dass etwas schief gelaufen ist.

Zum Beispiel `tf.image.adjust_contrast` Funktion [tf.image.adjust_contrast raw_ops.AdjustContrastv2,](https://www.tensorflow.org/api_docs/python/tf/raw_ops#.AdjustContrastv2) die einen Farbverlauf haben könnte, der Farbverlauf jedoch nicht implementiert ist:


In [ ]:
image = tf.Variable([[[0.5, 0.0, 0.0]]])
delta = tf.Variable(0.1)

with tf.GradientTape() as tape:
  new_image = tf.image.adjust_contrast(image, delta)

try:
  print(tape.gradient(new_image, [image, delta]))
  assert False   # This should not happen.
except LookupError as e:
  print(f'{type(e).__name__}: {e}')


If you need to differentiate through this op you'll either need to implement the gradient and register it (using `tf.RegisterGradient`), or re-implement the function using other ops.

## Nullen statt keine

In einigen Fällen wäre es zweckmäßig, für nicht verbundene Verläufe 0 anstelle von `None` zu erhalten. Mit dem Argument `unconnected_gradients` können Sie entscheiden, was zurückgegeben werden soll, wenn Sie nicht verbundene Verläufe haben:

In [ ]:
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))